In [4]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import torch 
torch.cuda.empty_cache()
torch.cuda.device_count()

3

In [1]:
import os
import torch
from datasets import load_dataset, load_from_disk
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
import evaluate
from typing import List, Dict

/blue/cai6307/y.khan/conda/envs/tabllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-02 23:30:53.414979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 23:31:00.938690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:/opt/slurm/lib64:
2024-04-02 23:31:00.938718: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart d

In [2]:
train_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_train")
test_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_test")
validate_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_validate")

In [3]:
test_example_ids = set(test_df['example_id'])
validate_example_ids = set(validate_df['example_id'])
common_example_ids = test_example_ids.intersection(validate_example_ids)

test_df = test_df.filter(lambda example: example['example_id'] not in common_example_ids)
test_df

Filter: 100%|██████████| 500/500 [00:00<00:00, 3418.78 examples/s]


Dataset({
    features: ['row_ids', 'table', 'summary', 'query', 'example_id', 'coordinates', 'answers'],
    num_rows: 300
})

In [8]:
model="google/flan-t5-xxl"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSeq2SeqLM.from_pretrained(model, load_in_8bit=True, device_map="auto")

tokenizer_config.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 22.1MB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 23.5MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:00<00:00, 19.2MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<00:00, 13.9MB/s]
config.json: 100%|██████████| 674/674 [00:00<00:00, 4.13MB/s]
model.safetensors.index.json: 100%|██████████| 53.0k/53.0k [00:00<00:00, 212MB/s]
model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]
model-00001-of-00005.safetensors:   0%|          | 21.0M/9.45G [00:00<00:46, 205MB/s]
model-00001-of-00005.safetensors:   0%|          | 41.9M/9.45G [00:00<00:45, 206MB/s]
model-00001-of-00005.safetensors:   1%|          | 62.9M/9.45G [00:00<00:45, 206MB/s]
model-00001-of-00005.safetensors:   1%|          | 83.9M/9.45G [00:00<00:45, 207MB/s]
model-00001-of-00005.safetensors:   1%|          | 115M/9.45G [00:00<00:44, 209MB/s] 
model-00001-of-00005.safetensors:   1%|▏         | 136M/9.45

In [9]:
def tokenization_with_answer(examples):
    inputs = []
    
    task_prefix = "Given a query and a table, generate a summary that answers the query based on the information in the table: "

    for i, (query, table, summary) in enumerate(zip(examples['query'], examples['table'], examples['summary'])):
        flattened_table = flatten_table(table, i)
        input_text = f"{task_prefix} Table {flattened_table}. Query: {query}"

        inputs.append(input_text)
        
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True,padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=512, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"] 

    return model_inputs


def flatten_table(table: Dict, row_index: int) -> str:
    header = table.get('header', [])
    rows = table.get('rows', [])
    title = table.get('title', [])

    flattened_rows = []
    for i, row in enumerate(rows):
        row_text = f"Row {i}, " + ",".join([f"{col}:{val}" for col, val in zip(header, row)])
        flattened_rows.append("## "+row_text)

    flattened_table = f"Title: {' '.join(map(str, title))}" + " " + " ".join(flattened_rows)
    return flattened_table

tokenized_dataset_train = train_df.map(tokenization_with_answer, batched=True)
tokenized_dataset_test = test_df.map(tokenization_with_answer, batched=True)

processed_data_train = tokenized_dataset_train.remove_columns(['table','summary', 'row_ids', 'example_id', 'query'])
processed_data_test = tokenized_dataset_test.remove_columns(['table','summary', 'row_ids', 'example_id', 'query'])

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


Map: 100%|██████████| 300/300 [00:02<00:00, 130.76 examples/s]


In [10]:
processed_data_train = processed_data_train.shuffle().select(range(1024))
processed_data_test = processed_data_test.shuffle().select(range(200))

In [12]:
# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=64,
 target_modules=["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj","v_proj", "ln", "fc", "q", "v", "lm_head"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 19,453,952 || all params: 11,154,786,304 || trainable%: 0.1744000420072951


In [13]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)

    rouge = evaluate.load('rouge')

    # Compute ROUGE scores
    rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

    return rouge_results

output_dir="lora-flan-t5-xxl"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    save_strategy = "no",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    learning_rate=2e-4, 
    num_train_epochs = 20,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=-1,
    logging_steps=25,
    weight_decay=0.001,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed_data_train,
    eval_dataset=processed_data_test,
    compute_metrics=metric_fn,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
